<a href="https://colab.research.google.com/github/praneshnikhar/ZenML/blob/main/Pipelines1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ML Pipelines with ZenML**


In [ ]:
%pip install "zenml[server]"


In [ ]:
!zenml integration install sklearn -y

⠴ Installing integrations...


In [ ]:
%pip install pyparsing==2.4.2
import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [ ]:
NGROK_TOKEN= "*************************"

In [ ]:
from zenml.environment import Environment
if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!rm -rf .zen
!zenml init

Initializing the ZenML global configuration version to 0.84.0
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /co

Experimentation ML code

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
  digits = load_digits()
  X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, shuffle = False)
  model = SVC(gamma = 0.001)
  model.fit(X_train , y_train)
  test_acc = model.score(X_test, y_test)
  print(f"Test accuracy: {test_acc}")

train_test()

Test accuracy: 0.9583333333333334


turning this into ML pipelines with ZenML

In [ ]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[Annotated[np.ndarray, "X_train"],
                        Annotated[np.ndarray, "X_test"],
                        Annotated[np.ndarray, "y_train"],
                        Annotated[np.ndarray, "y_test"]]:
    digits = load_digits()
    data= digits.images.reshape((len(digits.images),-1))
    X_train, X_test, y_train ,y_test= train_test_split(
        data,digits.target, test_size = 0.2, shuffle= False
    )
    return X_train , X_test, y_train , y_test

@step
def svc_trainer(
    X_train:np.ndarray,
    y_train:np.ndarray,
) -> ClassifierMixin:
    model = SVC(gamma = 0.001)
    model.fit(X_train , y_train)
    return model

@step
def evaluator(
    X_test:np.ndarray,
    y_test:np.ndarray,
    model: ClassifierMixin,
) -> float:
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    return test_acc

In [ ]:
from zenml import pipeline
@pipeline
def digits_pipeline():
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train, y_train)
    test_acc = evaluator(X_test, y_test, model)


Running ZenMl pipeline

In [ ]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step importer has started.
Step importer has finished in 12.020s.
Step svc_trainer has started.
Step svc_trainer has finished in 2.583s.
Step evaluator has started.
[evaluator] Test accuracy: 0.9583333333333334
Step evaluator has finished in 0.399s.
Pipeline run has finished in 19.434s.


In [ ]:
from zenml.environment import Environment
def start_zenml_dashboard(port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok
    public_url = ngrok.connect(port)
    print(rf"In Colab, use this URL instead: {public_url}!")

    !zenml up --blocking --port {port}
  else:
    !zenml up --port{port}
start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-07d1460d-2359-4c4f-9c96-3995018e8671


Opening tunnel named: http-8237-07d1460d-2359-4c4f-9c96-3995018e8671


INFO:pyngrok.process.ngrok:t=2025-07-21T17:13:31+0000 lvl=info msg="no configuration paths supplied"


t=2025-07-21T17:13:31+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2025-07-21T17:13:31+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

INFO:pyngrok.process.ngrok:t=2025-07-21T17:13:31+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]


t=2025-07-21T17:13:31+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]


ERROR:pyngrok.process.ngrok:t=2025-07-21T17:13:31+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: sample_data\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


t=2025-07-21T17:13:31+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: sample_data\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: sample_data\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.